This script is used for:
- training a model based on linear regression

**How to launch this Jupyter notebook**:   
```bash
execcasper -A your_project -l gpu_type=v100 -l walltime=06:00:00 -l select=1:ncpus=18:mpiprocs=36:ngpus=1:mem=300GB
bash aws_urban_env.sh
```

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import xarray as xr
import gc
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import joblib
import statsmodels.api as sm

start_year = "2006"
end_year = "2015"
urban_LE_nc_path = "/glade/scratch/zhonghua/urban_params/urban_LE/"
parquet_save_path = "/glade/scratch/zhonghua/urban_params/urban_LE_random_split/"
urban_surf_path = "/glade/scratch/zhonghua/urban_params/urban_surface.parquet.gzip"
model_name = "LR_climate"

In [2]:
fd = {
    "label":"TREFMXAV_U",
    "CAM": ['FLNS','FSNS','PRECT','PRSN','QBOT','TREFHT','UBOT','VBOT'],
    "surf":['CANYON_HWR','EM_IMPROAD','EM_PERROAD','EM_ROOF','EM_WALL', 
            'HT_ROOF','THICK_ROOF','THICK_WALL','T_BUILDING_MAX','T_BUILDING_MIN',
            'WTLUNIT_ROOF','WTROAD_PERV','NLEV_IMPROAD','PCT_URBAN',
            'ALB_IMPROAD','ALB_PERROAD','ALB_ROOF','ALB_WALL',
            'TK_ROOF','TK_WALL','CV_ROOF','CV_WALL',
            'TK_IMPROAD_0','CV_IMPROAD_0','TK_IMPROAD_1','CV_IMPROAD_1'],
    "loc":["lat","lon"]
}

def get_merge_member(start_year, end_year, parquet_save_path):
    df_tmp_ls = []
    for member_id in tqdm(range(3, 34)):
        member = (str(member_id).zfill(3))
        df_tmp_ls.append(pd.read_parquet(parquet_save_path + "train/" + member + "_"\
                            + start_year + "_" + end_year + ".parquet.gzip", engine="fastparquet"))
    return pd.concat(df_tmp_ls)

# load data
urban_LE = get_merge_member(start_year, end_year, parquet_save_path)
urban_surf = pd.read_parquet(urban_surf_path, engine="fastparquet").reset_index()

100%|███████████████████████████████████████████| 31/31 [00:08<00:00,  3.49it/s]


In [3]:
%%time
# merge data
df = pd.merge(urban_LE, urban_surf, on = ["lat","lon"], how = "inner")
# check if we merge the data successfully
assert urban_LE.shape[0] == df.shape[0]

del urban_LE, urban_surf
gc.collect()

CPU times: user 6.76 s, sys: 1.86 s, total: 8.62 s
Wall time: 8.61 s


21

## CAM + surf + loc

In [4]:
%%time
feature_ls = fd["CAM"]+fd["surf"]+fd["loc"]

X_ori = df[feature_ls]
y = df[fd["label"]]

scaler = StandardScaler()
scaler.fit(X_ori)
X_scaler = scaler.transform(X_ori)

X = sm.add_constant(X_scaler)
reg = sm.OLS(y, X).fit()

# ======== trianing performance ========
print("model performance using training data:")
y_pred = reg.predict(X)
print("root mean square error:", 
      mean_squared_error(y_true=y, y_pred=y_pred, squared=False))
print("mean_absolute_error:", mean_absolute_error(y_true=y, y_pred=y_pred))
print("r2:", r2_score(y_true=y, y_pred=y_pred))

model performance using training data:
root mean square error: 1.8990062595561652
mean_absolute_error: 1.4601806329528033
r2: 0.9696765004797875
CPU times: user 1min 31s, sys: 12.4 s, total: 1min 44s
Wall time: 1min 44s


In [5]:
def get_test_data(member, start_year, end_year, urban_LE_nc_path, parquet_save_path):
    # convert the time to datetime format
    ds_urban_LE = xr.open_dataset(urban_LE_nc_path+member+"_"+start_year+"_"+end_year+".nc")
    ds_urban_LE = ds_urban_LE.assign_coords(time = ds_urban_LE.indexes['time'].to_datetimeindex())
    df = ds_urban_LE.to_dataframe()
    
    df_train = pd.read_parquet(parquet_save_path + "train/" + member + "_"\
                               + start_year + "_" + end_year + ".parquet.gzip", engine="fastparquet")  
    
    del ds_urban_LE
    gc.collect()
    
    # remove missing value based on urban temperature
    df_final = df[~np.isnan(df["TREFMXAV_U"])].reset_index()
    df_final["member"] = member
    
    # get testing data based on the saved training data
    df_test = df_final.drop(df_train.index)
    return df_test

def get_pred(start_year, end_year, model_name, member, urban_LE_nc_path, parquet_save_path, reg):
    # ====== get data =======
    df_test = get_test_data(member, start_year, end_year, urban_LE_nc_path, parquet_save_path)
    urban_surf = pd.read_parquet(urban_surf_path, engine="fastparquet").reset_index()
    test = pd.merge(df_test, urban_surf, on = ["lat","lon"], how = "inner")
    assert df_test.shape[0] == test.shape[0] #check if we merged successfully
    del df_test, urban_surf
    gc.collect()
    # ====== pred and save=======
    print(feature_ls)
    
    X_test_scaler = scaler.transform(test[feature_ls])
    X_test = sm.add_constant(X_test_scaler)
    
    test["y_pred"] = reg.predict(X_test)
    pred_save_loc = parquet_save_path + "eval/" + model_name + "/CAM_surf_loc_"\
               + member + "_" + start_year + "_" + end_year + ".parquet.gzip"
    print("data loc:",pred_save_loc)
    test[["time","lat","lon","TREFMXAV_U","y_pred"]].to_parquet(pred_save_loc,
                                                                compression="gzip", engine="pyarrow")

In [6]:
%%time

start_year = "2061"
end_year = "2070"
for member_id in range(int("003"), int("034")):
    member = (str(member_id).zfill(3))
    get_pred(start_year, end_year, model_name, member, urban_LE_nc_path, parquet_save_path, reg)

['FLNS', 'FSNS', 'PRECT', 'PRSN', 'QBOT', 'TREFHT', 'UBOT', 'VBOT', 'CANYON_HWR', 'EM_IMPROAD', 'EM_PERROAD', 'EM_ROOF', 'EM_WALL', 'HT_ROOF', 'THICK_ROOF', 'THICK_WALL', 'T_BUILDING_MAX', 'T_BUILDING_MIN', 'WTLUNIT_ROOF', 'WTROAD_PERV', 'NLEV_IMPROAD', 'PCT_URBAN', 'ALB_IMPROAD', 'ALB_PERROAD', 'ALB_ROOF', 'ALB_WALL', 'TK_ROOF', 'TK_WALL', 'CV_ROOF', 'CV_WALL', 'TK_IMPROAD_0', 'CV_IMPROAD_0', 'TK_IMPROAD_1', 'CV_IMPROAD_1', 'lat', 'lon']
data loc: /glade/scratch/zhonghua/urban_params/urban_LE_random_split/eval/LR_climate/CAM_surf_loc_003_2061_2070.parquet.gzip
['FLNS', 'FSNS', 'PRECT', 'PRSN', 'QBOT', 'TREFHT', 'UBOT', 'VBOT', 'CANYON_HWR', 'EM_IMPROAD', 'EM_PERROAD', 'EM_ROOF', 'EM_WALL', 'HT_ROOF', 'THICK_ROOF', 'THICK_WALL', 'T_BUILDING_MAX', 'T_BUILDING_MIN', 'WTLUNIT_ROOF', 'WTROAD_PERV', 'NLEV_IMPROAD', 'PCT_URBAN', 'ALB_IMPROAD', 'ALB_PERROAD', 'ALB_ROOF', 'ALB_WALL', 'TK_ROOF', 'TK_WALL', 'CV_ROOF', 'CV_WALL', 'TK_IMPROAD_0', 'CV_IMPROAD_0', 'TK_IMPROAD_1', 'CV_IMPROAD_1', 'la

data loc: /glade/scratch/zhonghua/urban_params/urban_LE_random_split/eval/LR_climate/CAM_surf_loc_017_2061_2070.parquet.gzip
['FLNS', 'FSNS', 'PRECT', 'PRSN', 'QBOT', 'TREFHT', 'UBOT', 'VBOT', 'CANYON_HWR', 'EM_IMPROAD', 'EM_PERROAD', 'EM_ROOF', 'EM_WALL', 'HT_ROOF', 'THICK_ROOF', 'THICK_WALL', 'T_BUILDING_MAX', 'T_BUILDING_MIN', 'WTLUNIT_ROOF', 'WTROAD_PERV', 'NLEV_IMPROAD', 'PCT_URBAN', 'ALB_IMPROAD', 'ALB_PERROAD', 'ALB_ROOF', 'ALB_WALL', 'TK_ROOF', 'TK_WALL', 'CV_ROOF', 'CV_WALL', 'TK_IMPROAD_0', 'CV_IMPROAD_0', 'TK_IMPROAD_1', 'CV_IMPROAD_1', 'lat', 'lon']
data loc: /glade/scratch/zhonghua/urban_params/urban_LE_random_split/eval/LR_climate/CAM_surf_loc_018_2061_2070.parquet.gzip
['FLNS', 'FSNS', 'PRECT', 'PRSN', 'QBOT', 'TREFHT', 'UBOT', 'VBOT', 'CANYON_HWR', 'EM_IMPROAD', 'EM_PERROAD', 'EM_ROOF', 'EM_WALL', 'HT_ROOF', 'THICK_ROOF', 'THICK_WALL', 'T_BUILDING_MAX', 'T_BUILDING_MIN', 'WTLUNIT_ROOF', 'WTROAD_PERV', 'NLEV_IMPROAD', 'PCT_URBAN', 'ALB_IMPROAD', 'ALB_PERROAD', 'ALB_ROOF'

data loc: /glade/scratch/zhonghua/urban_params/urban_LE_random_split/eval/LR_climate/CAM_surf_loc_032_2061_2070.parquet.gzip
['FLNS', 'FSNS', 'PRECT', 'PRSN', 'QBOT', 'TREFHT', 'UBOT', 'VBOT', 'CANYON_HWR', 'EM_IMPROAD', 'EM_PERROAD', 'EM_ROOF', 'EM_WALL', 'HT_ROOF', 'THICK_ROOF', 'THICK_WALL', 'T_BUILDING_MAX', 'T_BUILDING_MIN', 'WTLUNIT_ROOF', 'WTROAD_PERV', 'NLEV_IMPROAD', 'PCT_URBAN', 'ALB_IMPROAD', 'ALB_PERROAD', 'ALB_ROOF', 'ALB_WALL', 'TK_ROOF', 'TK_WALL', 'CV_ROOF', 'CV_WALL', 'TK_IMPROAD_0', 'CV_IMPROAD_0', 'TK_IMPROAD_1', 'CV_IMPROAD_1', 'lat', 'lon']
data loc: /glade/scratch/zhonghua/urban_params/urban_LE_random_split/eval/LR_climate/CAM_surf_loc_033_2061_2070.parquet.gzip
CPU times: user 25min 53s, sys: 7min 32s, total: 33min 26s
Wall time: 34min 9s


In [ ]:
""" # evaluate
df_load = pd.read_parquet(parquet_save_path + "eval/" + model_name + "/CAM_surf_loc_"\
               + member + "_" + start_year + "_" + end_year + ".parquet.gzip", engine="pyarrow")
print("root mean square error:", 
      mean_squared_error(y_true=df_load["TREFMXAV_U"], y_pred=df_load["y_pred"], squared=False))
print("mean_absolute_error:", mean_absolute_error(y_true=df_load["TREFMXAV_U"], y_pred=df_load["y_pred"]))
print("r2:", r2_score(y_true=df_load["TREFMXAV_U"], y_pred=df_load["y_pred"]))
"""

## CAM

In [4]:
%%time
feature_ls = fd["CAM"]

X_ori = df[feature_ls]
y = df[fd["label"]]

scaler = StandardScaler()
scaler.fit(X_ori)
X_scaler = scaler.transform(X_ori)

X = sm.add_constant(X_scaler)
reg = sm.OLS(y, X).fit()

# ======== trianing performance ========
print("model performance using training data:")
y_pred = reg.predict(X)
print("root mean square error:", 
      mean_squared_error(y_true=y, y_pred=y_pred, squared=False))
print("mean_absolute_error:", mean_absolute_error(y_true=y, y_pred=y_pred))
print("r2:", r2_score(y_true=y, y_pred=y_pred))

def get_test_data(member, start_year, end_year, urban_LE_nc_path, parquet_save_path):
    # convert the time to datetime format
    ds_urban_LE = xr.open_dataset(urban_LE_nc_path+member+"_"+start_year+"_"+end_year+".nc")
    ds_urban_LE = ds_urban_LE.assign_coords(time = ds_urban_LE.indexes['time'].to_datetimeindex())
    df = ds_urban_LE.to_dataframe()
    
    df_train = pd.read_parquet(parquet_save_path + "train/" + member + "_"\
                               + start_year + "_" + end_year + ".parquet.gzip", engine="fastparquet")  
    
    del ds_urban_LE
    gc.collect()
    
    # remove missing value based on urban temperature
    df_final = df[~np.isnan(df["TREFMXAV_U"])].reset_index()
    df_final["member"] = member
    
    # get testing data based on the saved training data
    df_test = df_final.drop(df_train.index)
    return df_test

def get_pred(start_year, end_year, model_name, member, urban_LE_nc_path, parquet_save_path, reg):
    # ====== get data =======
    df_test = get_test_data(member, start_year, end_year, urban_LE_nc_path, parquet_save_path)
    urban_surf = pd.read_parquet(urban_surf_path, engine="fastparquet").reset_index()
    test = pd.merge(df_test, urban_surf, on = ["lat","lon"], how = "inner")
    assert df_test.shape[0] == test.shape[0] #check if we merged successfully
    del df_test, urban_surf
    gc.collect()
    # ====== pred and save=======
    print(feature_ls)
    
    X_test_scaler = scaler.transform(test[feature_ls])
    X_test = sm.add_constant(X_test_scaler)
    
    test["y_pred"] = reg.predict(X_test)
    pred_save_loc = parquet_save_path + "eval/" + model_name + "/CAM_"\
               + member + "_" + start_year + "_" + end_year + ".parquet.gzip"
    print("data loc:",pred_save_loc)
    test[["time","lat","lon","TREFMXAV_U","y_pred"]].to_parquet(pred_save_loc,
                                                                compression="gzip", engine="pyarrow")
    
start_year = "2061"
end_year = "2070"
for member_id in range(int("003"), int("034")):
    member = (str(member_id).zfill(3))
    get_pred(start_year, end_year, model_name, member, urban_LE_nc_path, parquet_save_path, reg)

model performance using training data:
root mean square error: 1.930452790294261
mean_absolute_error: 1.4797408730962052
r2: 0.9686639033377242
['FLNS', 'FSNS', 'PRECT', 'PRSN', 'QBOT', 'TREFHT', 'UBOT', 'VBOT']
data loc: /glade/scratch/zhonghua/urban_params/urban_LE_random_split/eval/LR_climate/CAM_003_2061_2070.parquet.gzip
['FLNS', 'FSNS', 'PRECT', 'PRSN', 'QBOT', 'TREFHT', 'UBOT', 'VBOT']
data loc: /glade/scratch/zhonghua/urban_params/urban_LE_random_split/eval/LR_climate/CAM_004_2061_2070.parquet.gzip
['FLNS', 'FSNS', 'PRECT', 'PRSN', 'QBOT', 'TREFHT', 'UBOT', 'VBOT']
data loc: /glade/scratch/zhonghua/urban_params/urban_LE_random_split/eval/LR_climate/CAM_005_2061_2070.parquet.gzip
['FLNS', 'FSNS', 'PRECT', 'PRSN', 'QBOT', 'TREFHT', 'UBOT', 'VBOT']
data loc: /glade/scratch/zhonghua/urban_params/urban_LE_random_split/eval/LR_climate/CAM_006_2061_2070.parquet.gzip
['FLNS', 'FSNS', 'PRECT', 'PRSN', 'QBOT', 'TREFHT', 'UBOT', 'VBOT']
data loc: /glade/scratch/zhonghua/urban_params/urban